# Búsqueda de autores en Orcid
See [search-orcid-registry](http://members.orcid.org/api/tutorial/search-orcid-registry)

ORCID: https://github.com/ORCID/python-orcid

In [1]:
%%writefile drive.cfg
[FILES]
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs 

Overwriting drive.cfg


In [2]:
import requests
import pandas as pd
import wosplus as wp

In [3]:
bdi=wp.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [4]:
bdi[:2]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes


In [5]:
# From https://github.com/ORCID/python-orcid
import orcid
api = orcid.PublicAPI('APP-R8J5P57TTE983P54', 'c6e1dd48-80d3-4dd9-9336-fc41728ac8fe', sandbox=False)
#token = api.get_token_from_authorization_code(authorization_code,
#                                              redirect_uri)


In [6]:
search_token = api.get_search_token_from_orcid()

In [7]:
search_token

'c2d0d987-d10d-4108-a13e-1064e074cd2f'

https://pub.orcid.org

In [8]:
HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}

In [9]:
r=requests.get('https://pub.orcid.org/v2.1/search/?q=newman', headers=HEADERS_orcid)

In [10]:
pd.read_json(r.text)

,num-found,result
0,876,"{'orcid-identifier': {'path': '0000-0002-7182-6999', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-7182-6999'}}"
1,876,"{'orcid-identifier': {'path': '0000-0002-5662-263X', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-5662-263X'}}"
2,876,"{'orcid-identifier': {'path': '0000-0002-0675-9611', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-0675-9611'}}"
3,876,"{'orcid-identifier': {'path': '0000-0003-1948-0385', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-1948-0385'}}"
4,876,"{'orcid-identifier': {'path': '0000-0003-4664-308X', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-4664-308X'}}"
5,876,"{'orcid-identifier': {'path': '0000-0003-4701-7523', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-4701-7523'}}"
6,876,"{'orcid-identifier': {'path': '0000-0002-3422-3523', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-3422-3523'}}"
7,876,"{'orcid-identifier': {'path': '0000-0002-5876-4498', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-5876-4498'}}"
8,876,"{'orcid-identifier': {'path': '0000-0001-8875-0655', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0001-8875-0655'}}"
9,876,"{'orcid-identifier': {'path': '0000-0003-0534-8914', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-0534-8914'}}"


In [78]:
1000/60

16.666666666666668

In [11]:
import pandas as pd
import time
import random
def search_orcid(q='family-name:Taylor+AND+given-names:Michael',search_token=''):
    '''
    Search API orcid in json for query q: (see examples here: http://members.orcid.org/api/tutorial/search-orcid-registry)
    Example: 
    q=family-name:Taylor+AND+given-names:Michael
    '''
    HEADERS_orcid={'Content-type': 'application/vnd.orcid+json',
    'Access token': search_token}
    r=requests.get('https://pub.orcid.org/v2.1/search/?q={}'.format(q),
                   headers=HEADERS_orcid)
    
    # wait for next request
    time.sleep(random.randint(1,3))
    if r.status_code==200:
        return pd.read_json(r.text)
    else:
        return pd.DataFrame()

## Formulas de búsqueda

In [12]:
search_token

'c2d0d987-d10d-4108-a13e-1064e074cd2f'

In [19]:
df=search_orcid('family-name:Restrepo+AND+given-names:Diego+AND+affiliation-org-name:Universidad+de+Antioquia',
                search_token)#+AND+given-names:Diego')#+AND+affiliation-org-name:Universidad+de+Antioquia')

In [20]:
df

,num-found,result
0,6,"{'orcid-identifier': {'path': '0000-0001-6455-5564', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0001-6455-5564'}}"
1,6,"{'orcid-identifier': {'path': '0000-0002-9054-3847', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-9054-3847'}}"
2,6,"{'orcid-identifier': {'path': '0000-0003-3575-1253', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-3575-1253'}}"
3,6,"{'orcid-identifier': {'path': '0000-0003-0432-9930', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-0432-9930'}}"
4,6,"{'orcid-identifier': {'path': '0000-0003-0639-0292', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-0639-0292'}}"
5,6,"{'orcid-identifier': {'path': '0000-0001-5649-6429', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0001-5649-6429'}}"


In [21]:
affiliation='Universidad de Antioquia'

In [22]:
affiliation.replace(' ','+')

'Universidad+de+Antioquia'

In [92]:
apellido='Restrepo'
nombre='Diego'
affiliation='Universidad de Antioquia'

In [93]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [94]:
df

,num-found,result
0,6,"{'orcid-identifier': {'path': '0000-0001-6455-5564', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0001-6455-5564'}}"
1,6,"{'orcid-identifier': {'path': '0000-0002-9054-3847', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-9054-3847'}}"
2,6,"{'orcid-identifier': {'path': '0000-0003-3575-1253', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-3575-1253'}}"
3,6,"{'orcid-identifier': {'path': '0000-0003-0432-9930', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-0432-9930'}}"
4,6,"{'orcid-identifier': {'path': '0000-0003-0639-0292', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-0639-0292'}}"
5,6,"{'orcid-identifier': {'path': '0000-0001-5649-6429', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0001-5649-6429'}}"


Además de afiliación:
1. Nombre completo
2. Nombre primer apellido
3. Segundo nombre primer appellido

In [53]:
bdi[:1]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación


In [65]:
bdi

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO,Primer_NOMBRE,Segundo_NOMBRE
0,71621971,Bernardo Maria Bustamante Cardona,Bernardo Maria,Bustamante,Cardona,Facultad de Artes,Departamento de Artes Visuales,Arteducación,Bernardo,Maria
1,71652139,Carlos Mario Jaramillo Ramirez,Carlos Mario,Jaramillo,Ramirez,Facultad de Artes,Departamento de Artes Visuales,DIDARTE. Didáctica de las artes,Carlos,Mario
2,43613390,Isabel Cristina Restrepo Acevedo,Isabel Cristina,Restrepo,Acevedo,Facultad de Artes,Departamento de Artes Visuales,Hipertrópico: Convergencia entre Arte y Tecnología,Isabel,Cristina
3,43986489,Alexandra Milena Tabares Garcia,Alexandra Milena,Tabares,Garcia,Facultad de Artes,Departamento de Artes Visuales,Hipertrópico: Convergencia entre Arte y Tecnología,Alexandra,Milena
4,15257718,Bernardo Barragan Castrillon,Bernardo,Barragan,Castrillon,Facultad de Artes,Departamento de Artes Visuales,Historia de la Práctica Pedagógica en Colombia,Bernardo,
5,18522153,Daniel Fernando Gomez Naranjo,Daniel Fernando,Gomez,Naranjo,Facultad de Artes,Departamento de Artes Visuales,Sin Grupo Asociado,Daniel,Fernando
6,43681505,Morelia Del Socorro Mesa Perez,Morelia Del Socorro,Mesa,Perez,Facultad de Artes,Departamento de Artes Visuales,Sin Grupo Asociado,Morelia,Del
7,6893516,Agustin Alonso Parra Grondona,Agustin Alonso,Parra,Grondona,Facultad de Artes,Departamento de Artes Visuales,Sin Grupo Asociado,Agustin,Alonso
8,70121904,Jorge Mario Villada Velez,Jorge Mario,Villada,Velez,Facultad de Artes,Departamento de Artes Visuales,Sin Grupo Asociado,Jorge,Mario
9,71685606,Romel Toro Carvajal,Romel,Toro,Carvajal,Facultad de Artes,Departamento de Artes Visuales,Sin Grupo Asociado,Romel,


In [37]:
i=0

In [41]:
bdi.loc[i,'NOMBRES'].strip()

'Bernardo Maria'

In [66]:
bdi.loc[4,'Segundo_NOMBRE'].strip()

''

In [44]:
bdi.loc[i,'PRIMER APELLIDO']+' '+bdi.loc[i,'SEGUNDO APELLIDO']

'Bustamante Cardona'

In [68]:
if 'ff':
    print(1)

1


In [70]:
if bdi.loc[1,'Segundo_NOMBRE'].strip():
    print(1)

1


In [80]:
df.loc[[0]]

,num-found,result
0,6,"{'orcid-identifier': {'path': '0000-0001-6455-5564', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0001-6455-5564'}}"


In [127]:
dft=pd.DataFrame()
affiliation='Universidad de Antioquia'
bdi['Primer_NOMBRE']=bdi.NOMBRES.str.split().str[0]
bdi['Segundo_NOMBRE']=bdi.NOMBRES.str.split().str[1].fillna('')
for i in bdi.index:
    df=pd.DataFrame()
    print(i)
    nombre_completo=bdi.loc[i,'NOMBRES'].strip()
    nombre=bdi.loc[i,'Primer_NOMBRE'].strip()
    segundo_nombre=bdi.loc[i,'Segundo_NOMBRE'].strip()
    apellido=bdi.loc[i,'PRIMER APELLIDO'].strip()
    apellido_completo=apellido+' '+bdi.loc[i,'SEGUNDO APELLIDO'].strip()
    #1. Busqueda en completo
    df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
    if not df.empty:
        df['forma']=1
    
    if df.empty:
        df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
        if not df.empty:
            df['forma']=2

        if df.empty and segundo_nombre:
            df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  segundo_nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)
            if not df.empty:
                df['forma']=3        

    if i==5:
        break
                
    if df.empty:
        continue 
    else:
        #inentar verificar y si funciona:
        df['NOMBRE COMPLETO']=bdi.loc[i,'NOMBRE COMPLETO'].strip()
        dft=dft.append(df.loc[[0]]).reset_index(drop=True)
        


0
1
2
3
4
5


In [ ]:
bdi.loc[i,'NOMBRE COOMPLETO']

In [118]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [123]:
df

,num-found,result,forma
0,1,"{'orcid-identifier': {'path': '0000-0002-8235-3525', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-8235-3525'}}",1


In [120]:
if not df.empty:
        df.loc[i,'forma']=1

In [128]:
dft

,num-found,result,forma,NOMBRE COMPLETO
0,1,"{'orcid-identifier': {'path': '0000-0003-3253-1597', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-3253-1597'}}",3,Bernardo Maria Bustamante Cardona
1,2,"{'orcid-identifier': {'path': '0000-0003-4969-8457', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-4969-8457'}}",1,Carlos Mario Jaramillo Ramirez
2,1,"{'orcid-identifier': {'path': '0000-0002-8235-3525', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0002-8235-3525'}}",1,Isabel Cristina Restrepo Acevedo


In [90]:
df=search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

In [103]:
apellido,nombre

('Bustamante', 'Bernardo')

In [105]:
search_orcid('family-name:{}+AND+given-names:{}'.format(
                  apellido.replace(' ','+'), 
                  nombre.replace(' ','+') ) ,
                search_token)

,num-found,result


In [98]:
apellido_completo,nombre_completo

('Bustamante Cardona', 'Bernardo Maria')

In [99]:
search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido_completo.replace(' ','+'), 
                  nombre_completo.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

,num-found,result


In [106]:
search_orcid('family-name:{}+AND+given-names:{}+AND+affiliation-org-name:{}'.format(
                  apellido.replace(' ','+'), 
                  segundo_nombre.replace(' ','+'), affiliation.replace(' ','+') ) ,
                search_token)

,num-found,result
0,1,"{'orcid-identifier': {'path': '0000-0003-3253-1597', 'host': 'orcid.org', 'uri': 'https://orcid.org/0000-0003-3253-1597'}}"


In [91]:
df

,num-found,result


In [ ]:
bdi[:]

In [96]:
dft

,num-found,result,forma
0,NaN,NaN,3.0


In [51]:
df.empty

False

In [47]:
nombre_completo,apellido,apellido_completo

('Bernardo Maria', 'Bustamante', 'Bustamante Cardona')

In [50]:
nombre_completo.split()[0]

'Bernardo'

## Appendix

In [17]:
r=requests.get('http://docs.python-requests.org/en/master/')

In [23]:
print( r.text[:193] )


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml" lang="en">
  <head>
    
